In [ ]:
import os
import pretty_midi

for filename in os.listdir("test_dataset"):
    try:
        midi_data = pretty_midi.PrettyMIDI("test_dataset/" + filename)
    except:
        continue
    
    f = open('index_dataset/' + filename + '.txt', 'w')
    print(filename)
    
    #first find the song duration
    song_duration = midi_data.get_end_time()
    
    
    #find tempo 
    tempo = midi_data.estimate_tempo()
    print("Tempo: " + str(tempo))
    #f.write(str(tempo) + "\n")
    
    segment_duration = 15
    
    #in each segment in sliding window write the notes to a file
    for i in range(0, int(song_duration)-segment_duration):
        for note in midi_data.instruments[0].notes:
            if note.start > i and note.end < i + segment_duration:
                f.write(str(note.pitch) + " ")
        f.write("\n")
        
    f.close()
    


In [5]:
import os
import pretty_midi

for filename in os.listdir("test_dataset"):
    try:
        midi_data = pretty_midi.PrettyMIDI("test_dataset/" + filename)
    except:
        continue
    
    
    print("File: " + filename)
    
    #first find the song duration
    song_duration = midi_data.get_end_time()
    print("Song duration: " + str(song_duration))
    
    #find the number of notes in the song
    print("Number of notes: " + str(len(midi_data.instruments[0].notes)))
    
    print(midi_data.instruments[0].notes)
    
    segment_duration = 30
    
    f = open('index_dataset/' + filename + '.txt', 'w')
    
    #in each segment in sliding window write the notes to a file
    
    # for i in range(0, int(song_duration)-segment_duration):
    #     prev = 1
    #     for j in range(prev, len(midi_data.instruments[0].notes)):
    #         if midi_data.instruments[0].notes[i].start > i and midi_data.instruments[0].notes[i].end < i+segment_duration:
    #             #relative pitches should be written
    #             f.write(str(abs(midi_data.instruments[0].notes[i].pitch - midi_data.instruments[0].notes[i-1].pitch)) + " ")
    #         f.write("\n")
    #     else:
    #         prev = j
    #         continue
        
    # f.close()
    
    for i in range(0, int(song_duration) - segment_duration):
        prev = 1
        for j in range(prev, len(midi_data.instruments[0].notes)):
            note = midi_data.instruments[0].notes[j]
            if note.start > i and note.end < i + segment_duration:
                # relative pitches should be written
                f.write(str(abs(note.pitch - midi_data.instruments[0].notes[j - 1].pitch)) + " ")
        f.write("\n")
        prev = j + 1

    f.close()

File: The_Beatles_-_Don't_Bother_Me.mid
Song duration: 101.14275600000002
Number of notes: 395
[Note(start=0.000000, end=0.178571, pitch=38, velocity=95), Note(start=0.214286, end=0.571428, pitch=38, velocity=91), Note(start=0.642857, end=0.821428, pitch=33, velocity=82), Note(start=0.857142, end=1.035713, pitch=38, velocity=102), Note(start=1.071428, end=1.428570, pitch=38, velocity=89), Note(start=1.499999, end=1.678570, pitch=33, velocity=76), Note(start=1.714284, end=1.892855, pitch=38, velocity=92), Note(start=1.928570, end=2.285712, pitch=38, velocity=78), Note(start=2.357141, end=2.535712, pitch=33, velocity=92), Note(start=2.571426, end=2.749997, pitch=38, velocity=92), Note(start=2.785712, end=2.964283, pitch=38, velocity=84), Note(start=2.999997, end=3.178568, pitch=39, velocity=94), Note(start=3.214283, end=3.392854, pitch=39, velocity=79), Note(start=3.428568, end=3.607139, pitch=40, velocity=90), Note(start=3.642854, end=3.999996, pitch=40, velocity=90), Note(start=4.07142

In [8]:
import pretty_midi
midi = pretty_midi.PrettyMIDI("test_dataset/dataset_pop/Eminem_-_The_Real_Slim_Shady.mid")

for i in range(0,6):
    notes = midi.instruments[i].notes
    
    #save the notes in new midi file
    new_midi = pretty_midi.PrettyMIDI()
    for j in range(0,11):
        new_midi.instruments.append(pretty_midi.Instrument(j))
    for note in notes:
        new_midi.instruments[i].notes.append(note)
    new_midi.write('test_dataset/test/Words' + str(i) + '.mid')




In [3]:
from fastdtw import fastdtw
from dtw import utw, ldtw

def get_dtw_distance(input_pitches, segment_pitches):
    distance = ldtw(input_pitches, segment_pitches, 2)
    
    return distance

In [4]:
input_midi_data = pretty_midi.PrettyMIDI("input.mid")
input_pitches=[]
for i in range(1, len(input_midi_data.instruments[0].notes)):
    input_pitches.append(abs(input_midi_data.instruments[0].notes[i].pitch - input_midi_data.instruments[0].notes[i-1].pitch))

# for note in input_midi_data.instruments[0].notes:
#     input_pitches.append(note.pitch)
    
    
print("Input pitches: " + str(input_pitches))

top_5 = []

for filename in os.listdir("index_dataset"):
    smallest = 10000000
    best_segment = None
    f = open('index_dataset/' + filename, 'r')
    content = f.read()
    notes = content.split("\n")
    
    for note in notes:
        if note == '':
            continue
        #convert string to array of integers
        
        note = note.split(" ")
        #remove last element
        note.pop()
        note = list(map(int, note))
        distance = get_dtw_distance(input_pitches, note)
        if distance < smallest:
            smallest = distance
            best_segment = note
    
    print("File: " + filename)
    print("Distance: " + str(smallest))
    print("Segment: " + str(best_segment))        
    #top_5.append({"filename": filename, "distance": smallest, "segment": best_segment})
    
    
    f.close()
#top_5.sort(key=lambda x: x["distance"])
# for i in range(0, 5):
#     print("File: " + top_5[i]["filename"])
#     print("Distance: " + str(top_5[i]["distance"]))
#     print("Segment: " + str(top_5[i]["segment"]))
#     print("\n")
    
        

Input pitches: [0, 4, 0, 1, 0, 1, 0, 4, 2, 6, 0, 2, 0, 0, 2, 1, 2, 0, 1, 1, 9, 0, 0, 0, 2, 1, 4, 2, 3, 0, 0, 2, 22, 24, 2, 1, 1, 0, 0, 0]
File: The_Beatles_-_Don't_Bother_Me.mid.txt
Distance: 317.0
Segment: [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 5, 5, 0, 5, 5, 0, 0, 0, 0, 0, 0, 7, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 5, 0, 5, 8, 0, 5, 5, 0, 5, 5, 0, 5, 12, 0, 3, 0, 6, 0, 5, 5, 0, 5, 5, 0, 5, 12, 0, 3, 0, 10, 0, 5, 0, 5, 5, 0, 2, 0, 5, 5, 0, 5, 3, 0, 5, 5, 0, 5, 14, 0, 5, 5, 0, 2, 2, 1, 0, 5, 5, 0, 5, 3, 0, 5, 5, 0, 5, 0, 7]
File: The_Beatles_-_Do_You_Want_to_Know_a_Secret.mid.txt
Distance: 86.0
Segment: [7, 0, 4, 1, 1, 0, 5, 2, 1, 2, 2, 0, 4, 1, 1, 0, 5, 2, 1, 2, 2, 0, 4, 1, 1, 0, 6, 8, 4, 1, 1, 1, 1, 0, 5, 2, 1, 2, 2, 0]
File: The_Beatles_-_Drive_My_Car.mid.txt
Distance: 10000000
Segment: None
File: The_Beatles_-_Eight_Days_A_Week.mid.txt
Distance: 941.0
Segment: [8, 11, 3, 1, 7, 11, 11, 0, 8, 1, 7, 11, 0, 4, 1, 8, 11, 14, 11, 8, 11, 0, 11, 0,

In [ ]:
from elasticsearch import Elasticsearch

es = Elasticsearch(
    "https://localhost:9200/",
    ca_certs="C:\Elastic\elasticsearch-8.11.1\config\certs\http_ca.crt",
    basic_auth=("elastic", "PgrlijTLFLyC9X3Y*pRa")
)

print(es.ping())


In [ ]:
#create index
mapping = {
    "mappings": {
        "properties": {
            'song': { 'type': 'text' },
            'notes': { 'type': 'text'},
            'tempo': { 'type': 'float'},
            'relative_pitches': { 'type': 'text'}
        }
    }
}

if(es.indices.exists(index="midi")):
    es.indices.delete(index="midi")

es.indices.create(index="midi", body=mapping, ignore=400)

In [ ]:
for filename in os.listdir("index_dataset"):
    f = open("index_dataset/" + filename, "r")
    content = f.read()
    
    #each line should be a document
    line = content.split("\n")
    for i in range(0, len(line)):
        if (line[i]== ""):
            continue
        doc = {
            "song": filename,
            "notes": line[i]
        }
        es.index(index="midi", body=doc)
    
    print(filename + " indexed")
    
    f.close()

In [ ]:
from fastdtw import fastdtw

def get_dtw_distance(input_pitches, segment_pitches):
    distance, path = fastdtw(input_pitches, segment_pitches)
    
    return distance

In [ ]:
input_midi_data = pretty_midi.PrettyMIDI("input.mid")
print(input_midi_data.estimate_tempo())

In [ ]:
#given a query mid file, return the most similar mid file

#use cosine similarity
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

input_midi_data = pretty_midi.PrettyMIDI("input.mid")
input_pitches=[]
for note in input_midi_data.instruments[0].notes:
    input_pitches.append(note.pitch)

query = {
        'query': {
            'function_score': {
                'functions': [
                    {
                        'script_score': {
                            'script': {
                                'source': 'fastdtw(params.query_pitches, doc["notes"])',
                                'params': {
                                    'query_pitches': input_pitches
                                }
                            }
                        }
                    }
                ],
                'score_mode': 'avg'
            }
        }
}

result = es.search(index='midi', body=query)
print(result['hits']['hits'][0:3]['_source']['song']) 



In [ ]:
#given a query mid file, return the most similar mid file

#use cosine similarity
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

input_midi_data = pretty_midi.PrettyMIDI("input.mid")
input_pitches=[]
for note in input_midi_data.instruments[0].notes:
    input_pitches.append(note.pitch)


dtw_script = {
        "script": {
            "source": "1.0 / (1.0 + get_dtw_distance(params.query_sequence, doc['features'].notes))",
            "lang": "painless",
        },
        "params": {"query_sequence": input_pitches},
    }

query = {
        "query": {
            "function_score": {
                "query": {"match_all": {}},
                "script_score": dtw_script
            }
        }
    }


results = es.search(index='midi', body=query)
print(results['hits']['hits'][:3])
